In [17]:
import sys
sys.path.insert(0, '../')

In [18]:
from typing import Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Subset
from torchvision import datasets

import albumentations as A
import albumentations.pytorch.transforms as AT

import tensorclan
import tensorclan.dataset as tcd
import tensorclan.model as tcm
import tensorclan.dataset as tcd
import tensorclan.dataset.augmentation as tca
import tensorclan.utils as tc_utils
import tensorclan.runner as tc_runner

In [19]:
%%writefile custom_model.yaml

name: TTC_EXAMPLE
log_dir: logs
chkpt_dir: checkpoint
use_checkpoints: true

device: GPU

model:
    name: Net

dataset:
    name: MNISTV2
    transforms: MNISTTransformsV2
    args:
        root: data
    loader_args:
        batch_size: 128
        num_workers: 2
        shuffle: True
        pin_memory: True

loss: CrossEntropyLoss

optimizer:
    type: SGD
    args:
        lr: 0.01
        momentum: 0.95

training:
    epochs: 5

Overwriting custom_model.yaml


In [20]:
@tcm.model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x

@tcd.dataset
class MNISTV2(tcd.BaseDataset):
    test_set: datasets.MNIST
    train_set: datasets.MNIST

    def __init__(self, root: str, transforms=None):
        self.data_dir = root
        self.transforms = transforms

    @staticmethod
    def split_dataset(dataset, transforms):

        train_set = datasets.MNIST(
            dataset.data_dir,
            train=True,
            download=True,
            transform=transforms.build_transforms(train=True),
        )

        test_set = datasets.MNIST(
            dataset.data_dir,
            train=False,
            download=True,
            transform=transforms.build_transforms(train=False)
        )
        return Subset(train_set, indices=range(0, len(train_set))), Subset(test_set, indices=range(0, len(test_set)))

    @staticmethod
    def plot_sample(sample):
        pass


@tca.augmentation
class MNISTTransformsV2(tca.BaseAugmentation):

    mean: Tuple[float] = (0.1307,)
    std: Tuple[float] = (0.3081,)

    def build_train(self):
        train_transforms = A.Compose([
            A.Normalize(mean=self.mean, std=self.std),
            AT.ToTensor()
        ])
        return train_transforms

    def build_test(self):
        test_transforms = A.Compose([
            A.Normalize(mean=self.mean, std=self.std),
            AT.ToTensor()
        ])

        return test_transforms

[ 2020-07-31 14:21:21,887 - tensorclan.model.model ] INFO: Registered Net as model
[ 2020-07-31 14:21:21,888 - tensorclan.dataset.dataset ] INFO: Registered MNISTV2 as dataset
[ 2020-07-31 14:21:21,890 - tensorclan.dataset.augmentation.augmentation ] INFO: Registered MNISTTransformsV2 as augmentation


In [21]:
config = tc_utils.load_config('custom_model.yaml')

In [22]:
runner = tc_runner.Runner(config=config)


[ 2020-07-31 14:21:21,958 - tensorclan.runner.runner ] INFO: => Now simply setup_train and then start_train your model


In [23]:
runner.setup_train()

[ 2020-07-31 14:21:22,017 - tensorclan.runner.runner ] INFO: => Config
[ 2020-07-31 14:21:22,019 - tensorclan.runner.runner ] INFO: {'chkpt_dir': 'checkpoint',
[ 2020-07-31 14:21:22,021 - tensorclan.runner.runner ] INFO:  'dataset': {'args': {'root': 'data'},
[ 2020-07-31 14:21:22,023 - tensorclan.runner.runner ] INFO:              'loader_args': {'batch_size': 128,
[ 2020-07-31 14:21:22,024 - tensorclan.runner.runner ] INFO:                              'num_workers': 2,
[ 2020-07-31 14:21:22,026 - tensorclan.runner.runner ] INFO:                              'pin_memory': True,
[ 2020-07-31 14:21:22,028 - tensorclan.runner.runner ] INFO:                              'shuffle': True},
[ 2020-07-31 14:21:22,029 - tensorclan.runner.runner ] INFO:              'name': 'MNISTV2',
[ 2020-07-31 14:21:22,030 - tensorclan.runner.runner ] INFO:              'transforms': 'MNISTTransformsV2'},
[ 2020-07-31 14:21:22,032 - tensorclan.runner.runner ] INFO:  'device': 'GPU',
[ 2020-07-31 14:21:22,0

In [24]:
runner.start_train()

[ 2020-07-31 14:21:22,113 - tensorclan.trainer.gpu_trainer ] INFO: => Training Started
[ 2020-07-31 14:21:22,115 - tensorclan.trainer.gpu_trainer ] INFO: Training the model for 5 epochs
[ 2020-07-31 14:21:22,115 - tensorclan.trainer.gpu_trainer ] INFO: => Training Epoch 0
[ 2020-07-31 14:21:26,290 - tensorclan.trainer.gpu_trainer ] INFO: loss: 0.6750475881514011, accuracy: 77.665
[ 2020-07-31 14:21:26,291 - tensorclan.trainer.gpu_trainer ] INFO: => Testing Epoch 0
[ 2020-07-31 14:21:27,078 - tensorclan.trainer.gpu_trainer ] INFO: loss: 0.10571440865721883 accuracy: 96.65
[ 2020-07-31 14:21:27,081 - tensorclan.trainer.gpu_trainer ] INFO: => Accuracy improved, saving best checkpoint ...
[ 2020-07-31 14:21:27,085 - tensorclan.trainer.gpu_trainer ] INFO: => Saving checkpoint ...
[ 2020-07-31 14:21:27,090 - tensorclan.trainer.gpu_trainer ] INFO: 

[ 2020-07-31 14:21:27,091 - tensorclan.trainer.gpu_trainer ] INFO: => Training Epoch 1
[ 2020-07-31 14:21:31,251 - tensorclan.trainer.gpu_trainer